In [44]:
import numpy as np
from glob import glob
from skimage.morphology import skeletonize_3d
from medutils.medutils import load_itk, save_itk
import SimpleITK as sitk
import itk
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.morphology import disk,diamond,rectangle,square,erosion,dilation,opening,closing,skeletonize
import pickle
from scipy.ndimage.morphology import binary_dilation, generate_binary_structure
import time

Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
Please use `generate_binary_structure` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.


In [14]:
def create_bin_sphere(arr_size, center, r):
    coords = np.ogrid[:arr_size[0], :arr_size[1], :arr_size[2]]
    distance = np.sqrt((coords[0] - center[0])**2 + (coords[1]-center[1])**2 + (coords[2]-center[2])**2) 
    return 1*(distance <= r)

In [15]:
arr_size = (3,3,3)
sphere_center = (1,1,1)
r=1
sphere = create_bin_sphere(arr_size,sphere_center, r)

In [16]:
sphere

array([[[0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]],

       [[0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]]])

In [2]:
fpath = "/data1/jjia/dataset/lung_function/iso1.5/SSc_patient_0068800_GcVessel.nii.gz"

In [39]:
fpath_ls = glob("/data1/jjia/dataset/lung_function/iso1.5/SSc_patient_???????.nii.gz")

In [40]:
len(fpath_ls)

314

In [51]:
for i in fpath_ls[1:2]:
    x, ori, sp = load_itk(i, require_ori_sp=True)
    vs_fpath = i.replace('.nii.gz', '_GcVessel.nii.gz')
    try:
        vs = load_itk(vs_fpath)
        for i in range(6):
            vs = binary_dilation(vs)
            save_itk(vs_fpath.replace('.nii.gz', f'_dilated_{i+1}.nii.gz'), vs, ori, sp)
    except:
        print('not found', i)

In [47]:
fpath_ls[0]

'/data1/jjia/dataset/lung_function/iso1.5/SSc_patient_7886110.nii.gz'

In [25]:
fpath_vs_ls = glob("/data1/jjia/dataset/lung_function/iso1.5/SSc_patient_???????_GcVessel.nii.gz")

In [26]:
len(fpath_vs_ls)

332

In [6]:
b, ori, sp = load_itk(fpath, require_ori_sp=True)

In [12]:
c =  (b, structure=struct1)

In [10]:
struct1 = generate_binary_structure(3, 3)

In [11]:
struct1

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]])

In [13]:
save_itk(fpath.replace('.nii.gz', 'dilated_fullycon.nii.gz'), c, ori, sp)

In [52]:
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.

    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst

    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

In [ ]:
def square_distance2(src, dst):
    """
    Calculate Euclid distance between each two points.

    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst

    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

In [54]:
import torch

In [63]:
src = torch.ones((1,2,3))
dst = torch.ones((1,2,3))

In [64]:
src.shape

torch.Size([1, 2, 3])

In [66]:
c = square_distance(src, dst)

In [68]:
a = torch.tensor([1,2,3])

In [71]:
c

tensor([[[0., 0.],
         [0., 0.]]])

In [74]:
d = (src - dst) ** 2

In [75]:
d.shape

torch.Size([1, 2, 3])